> Copyright 2025 Giovanni Squillero <<giovanni.squillero@polito.it>>  
> SPDX-License-Identifier: `0BSD`

In [1]:
import random
from itertools import accumulate
from functools import cache

import numpy as np
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt

In [2]:
def augmented(func):
    class _Augmnented:
        _log: list

        def __init__(self):
            self.clear()

        @property
        def log(self):
            return self._log[:]

        def clear(self):
            self._log = list()

        def plot(self, figsize=(14, 8)):
            best = list(accumulate(self._log, max))
            improvements = [(i, v) for i, v in list(enumerate(best)) if v > best[i - 1]]
            plt.figure(figsize=figsize)
            plt.scatter(range(len(self._log)), self._log, marker='.', color='lavender')
            plt.plot(range(len(best)), best, color='lightcoral')
            plt.scatter(
                [i for i, v in improvements],
                [v for i, v in improvements],
                marker='*',
                color='red',
            )

        @cache
        def __call__(self, *args, **kwargs):
            self._log.append(func(*args, **kwargs))
            return self.log[-1]

    return _Augmnented()

# 0-1 Multiple Knapsack Problem

see: [https://en.wikipedia.org/wiki/Knapsack_problem](https://en.wikipedia.org/wiki/Knapsack_problem)

In [3]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [9]:
NUM_ITEMS = 2000  # Esercizio con 100 items
DIMENSIONS = 50  # Esercizio con 5 dimensioni
MAX_STEPS = 200_000

WEIGHTS = np.random.randint(1, 50 + 1, size=(NUM_ITEMS, DIMENSIONS))
MAX_WEIGHTS = np.full(DIMENSIONS, NUM_ITEMS * 20)
VALUES = np.random.randint(1, 100 + 1, size=NUM_ITEMS)

In [10]:
def evaluate(knapsack):
    if all(np.sum(WEIGHTS[knapsack], axis=0) < MAX_WEIGHTS):
        return np.sum(VALUES[knapsack])
    else:
        return -1


@augmented
def fitness(knapsack):
    return evaluate(list(knapsack))

## Local search (hill-climbing) 
### Random strength tweak

In [11]:
def tweak(solution, p):
    solution = list(solution)
    i = random.randint(0, len(solution) - 1)
    solution[i] = not solution[i]
    while random.random() < p:
        i = random.randint(0, len(solution) - 1)
        solution[i] = not solution[i]
    return tuple(solution)

In [ ]:
tries = 0
fitness.clear()

for mutation_strength in [0, .1, .2, .4, .8]:
    best_solution = tuple(random.choice([True, False]) for _ in range(NUM_ITEMS))
    for tries in tqdm(range(MAX_STEPS)):
        solution = tweak(best_solution, mutation_strength)
        if fitness(solution) > fitness(best_solution):
            best_solution = solution[:]

    print(f"Solution found in {tries:,} tries: {evaluate(list(best_solution)):,}")
    #fitness.plot()

  0%|          | 0/200000 [00:00<?, ?it/s]

Solution found in 199,999 tries: 75,371


  0%|          | 0/200000 [00:00<?, ?it/s]

Solution found in 199,999 tries: 87,361


  0%|          | 0/200000 [00:00<?, ?it/s]

Solution found in 199,999 tries: 90,285


  0%|          | 0/200000 [00:00<?, ?it/s]

Solution found in 199,999 tries: 92,424


  0%|          | 0/200000 [00:00<?, ?it/s]